# Set-Up

In [1]:
!pip install langchain -q
!pip install langchain_chroma -q
!pip install langchain_community -q
!pip install langchain_groq -q
!pip install grandalf -q
!pip install numpy -q
!pip install pandas -q
!pip install sentence-transformers -q
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 5.4 MB/s eta 0:

In [2]:
import os

os.environ['GROQ_API_KEY'] = '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'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # annoying transformers warning

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Groq Llama API

In [4]:
youtube_data_path = '/content/drive/MyDrive/itmo-things/psycho_rag_hggg'

In [5]:
from langchain_groq import ChatGroq
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
rag_llm = ChatGroq(model="llama3-8b-8192")  # Used for RAG

<ipython-input-5-180b249cbc5d>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warni

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Vectorization

In [6]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
loader = DirectoryLoader(youtube_data_path, use_multithreading=True, loader_cls=TextLoader)
text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', ' ', ''],
    chunk_size=3000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

documents = loader.load_and_split(text_splitter=text_splitter)
vectorstore = Chroma.from_documents(documents, embedding=embed_model, collection_name="groq_rag")
retriever = vectorstore.as_retriever()
print(f"Documents indexed: {len(documents)}")

Documents indexed: 517


In [8]:
await retriever.ainvoke("How to fix my ADHD?")

[Document(id='9df87508-edb9-4c04-a3b7-7897c2f5c3a0', metadata={'source': '/content/drive/MyDrive/itmo-things/psycho_rag_hggg/Why ADHD is Linked with Addiction [HNje-HuIYdI].txt'}, page_content="things to do and none of them are easy and none of them are simple. At the top of the list is get sober. I'm sorry but you can't convince yourself anymore that drugs are like an acceptable thing to do if you've got ADHD. If you have ADHD that is not well controlled and it's negatively impacting your life, honestly the first thing to do is to get sober. Second thing to do is ideally work with a dual diagnosis clinician who can do therapy to help you work through shame as well as like do CBT around ADHD and stuff like that and can help you like get into the process of recovery. Third thing that you can do is a dopamine detox. So this is going to be really tough for people with ADHD because dopamine detox, the primary problem is essentially boredom. We have a video about dopamine detox that y'all c

# Rag

In [9]:
from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from typing import List, Dict

In [10]:
RAG_SYSTEM_PROMPT = """\
You are a psychological assistant for answering questions about mental health. \
You have knowledge base from which you're retrieving some context based on question. \
Use the following pieces of retrieved context given within delimiters to answer the human's questions.
```
{context}
```
Be very careful if you don't know the answer, it's dangerous to give bad answers, just say that you don't know.\
"""

RAG_HUMAN_PROMPT = "{input}"

RAG_PROMPT = ChatPromptTemplate.from_messages([
    ("system", RAG_SYSTEM_PROMPT),
    ("human", RAG_HUMAN_PROMPT)
])

def format_docs(docs: List[Document]):
    """Format the retrieved documents"""
    return "\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs, # Use retriever to retrieve docs from vectorstore -> format the documents into a string
        "input": RunnablePassthrough() # Propogate the 'input' variable to the next step
    }
    | RAG_PROMPT # format prompt with 'context' and 'input' variables
    | rag_llm # get response from LLM using the formatteed prompt
    | StrOutputParser() # Parse through LLM response to get only the string response

)

In [11]:
await rag_chain.ainvoke("How to be more social if I'm afraid of people?")

'What a great question! I totally understand that social anxiety can be overwhelming, but I\'m here to help you work through it.\n\nFirstly, acknowledge that social anxiety is common and valid. It\'s not about being shy or introverted, but rather a natural response to perceived threats or uncomfortable situations.\n\nTo start building social skills and overcoming anxiety, let\'s break it down into smaller, manageable steps. Here are some tips:\n\n1. **Identify your internal challenges**: Take time to reflect on what specifically makes you anxious in social situations. Is it fear of rejection, fear of being judged, or something else? Becoming aware of your anxiety triggers can help you prepare and develop strategies to cope.\n2. **Start small**: Begin with low-stakes interactions, like saying hello to a cashier or asking a question in a class. Gradually increase the intensity and duration of your social interactions.\n3. **Practice self-compassion**: Be kind to yourself when you make mi

# Class Implementation

In [57]:
from typing import List
from langchain_groq import ChatGroq
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema import HumanMessage, AIMessage, SystemMessage

class PsychoRag:
    def __init__(self, data_path: str):
        self.embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
        self.rag_llm = ChatGroq(model="llama3-8b-8192")

        self.memory = ConversationSummaryBufferMemory(
            return_messages=True,
            llm=self.rag_llm,
            max_token_limit=1000
        )
        self.question_max_tokens = 1000

        self.loader = DirectoryLoader(data_path, use_multithreading=True, loader_cls=TextLoader)
        self.text_splitter = RecursiveCharacterTextSplitter(
            separators=['\n\n', '\n', ' ', ''],
            chunk_size=2000,
            chunk_overlap=200,
            length_function=len,
            is_separator_regex=False,
        )
        self.documents = self.loader.load_and_split(text_splitter=self.text_splitter)
        self.vectorstore = Chroma.from_documents(self.documents, embedding=self.embed_model, collection_name="groq_rag")
        self.retriever = self.vectorstore.as_retriever()

        RAG_SYSTEM_PROMPT = """\
            You are a psychological assistant for answering questions about mental health. \
            You have knowledge base from which you're retrieving some context based on question. \

            Based on conversation history
            '''
            {history}
            '''

            Use the following pieces of retrieved context given within delimiters to answer the human's questions:
            '''
            {context}
            '''

            Be very careful if you don't know the answer, it's dangerous to give bad answers, just say that you don't know.\
            """

        RAG_HUMAN_PROMPT = """\
          {input}
          """

        self.RAG_PROMPT = ChatPromptTemplate.from_messages([
            ("system", RAG_SYSTEM_PROMPT),
            ("human", RAG_HUMAN_PROMPT)
        ])

        self.rag_chain = (
            {
                "context": RunnableLambda(func=lambda x: x['input']) | self.retriever | self.format_docs,
                "history": RunnableLambda(func=lambda x: self.memory.load_memory_variables({}).get("history", "")),
                "input": RunnablePassthrough()
            }
            | self.RAG_PROMPT
            | self.rag_llm
            | StrOutputParser()
        )

    def format_docs(self, docs: List[Document]):
        return "\n".join(doc.page_content for doc in docs)

    def preprocess_user_input(self, user_input: str) -> str:
        if len(user_input) > self.question_max_tokens:
            return user_input[: self.question_max_tokens]
        return user_input

    async def ask(self, user_input: str) -> str:
        user_input = self.preprocess_user_input(user_input)
        answer = await self.rag_chain.ainvoke({"input": user_input})

        self.memory.chat_memory.add_user_message(user_input)
        self.memory.chat_memory.add_ai_message(answer)
        return answer

    async def end_session(self):

        SUMMARY_PROMPT = """\
          You are a psychologist who just completed a session with a client (the user).
          Below is the entire conversation you had with the user.

          Conversation:
          '''
          {history}
          '''

          Based on the entire conversation:
          1. Summarize the user's main psychological concerns and issues that came up.
          2. Provide a few possible methods or strategies for the user to work on these issues.
          3. Maintain an empathetic, understanding tone.
          4. If there is insufficient information to conclude on something, mention that gently.
          """

        memory_variables = self.memory.load_memory_variables({})
        memory_history = memory_variables.get("history", "")

        if isinstance(memory_history, list):
            def format_message(msg):
                if isinstance(msg, HumanMessage):
                    role = "User"
                elif isinstance(msg, AIMessage):
                    role = "Assistant"
                elif isinstance(msg, SystemMessage):
                    role = "System"
                else:
                    role = "Unknown"
                return f"{role}: {msg.content}"

            memory_history = "\n".join(format_message(msg) for msg in memory_history)

        elif not isinstance(memory_history, str):
            raise ValueError(f"Expected history to be a string or list, got: {type(memory_history)}")

        if not memory_history.strip():
            return (
                "No conversation history found. Please engage in a conversation first, "
                "and I'll be happy to provide a summary based on the session."
            )

        report_prompt = ChatPromptTemplate.from_messages([
            ("system", SUMMARY_PROMPT.replace("{history}", memory_history))
        ])

        summary_chain = (
            {
                "input": RunnablePassthrough()
            }
            | report_prompt
            | self.rag_llm
            | StrOutputParser()
        )

        report = await summary_chain.ainvoke("")
        return report


In [58]:
rag_sys = PsychoRag(youtube_data_path)

In [59]:
answer = await rag_sys.ask("hello.")
print(answer)

Hello! It's great to hear from you. I sense that you're trying to break the ice and start a conversation. I'm happy to chat with you.

From our previous conversation, I remember that you were trying to find ways to deal with your scattered mind and social anxiety. You mentioned that you often feel like you're stuck in a rut and that your mind is always racing with thoughts.

If you're feeling comfortable, I'd love to continue exploring ways to help you manage your mental health. Would you like to talk more about what's been going on lately?


In [60]:
answer = await rag_sys.ask("What kind of meditation would you suggest for me?")

print(answer)

Based on our previous conversation, I sense that you're looking for a meditation technique that would help you control your emotions, thoughts, and mind. You've mentioned that you've been trying to meditate by sitting and thinking really fast, which might not be the most effective approach.

The technique that you're interested in, which involves wrapping a prayer box around your head and arms seven times, is based on a practice that a rabbi told you about. This practice is meant to help you control your thoughts and emotions, and if successful, lead you to being completely in control of your life.

However, I want to caution you that this technique might not be the most effective or scientifically-backed approach to meditation. As your psychological assistant, I'm concerned about the potential risks of using untested and unproven methods.

Instead, I would recommend exploring more established and evidence-based meditation techniques that can help you achieve your goals. These might in

In [61]:
answer = await rag_sys.ask("how to fix my ADHD?")

print(answer)

Based on our previous conversation and the context you've provided, I sense that you're looking for ways to manage your Attention Deficit Hyperactivity Disorder (ADHD). You've mentioned that you often feel like you're stuck in a rut and that your mind is always racing with thoughts.

Firstly, I want to acknowledge that ADHD is a neurodevelopmental disorder that affects individuals differently. It's essential to recognize that there is no one-size-fits-all solution to "fix" ADHD, but rather a combination of strategies and approaches that can help you manage its symptoms.

From our previous conversation, I've noticed that you've been trying to meditate by sitting and thinking really fast, which might not be the most effective approach. I want to caution you that untested and unproven methods might not be beneficial for your mental health.

Instead, I'd like to suggest some evidence-based strategies that may help you manage your ADHD symptoms:

1. **Structure and Routine**: Establishing a

In [62]:
report = await rag_sys.end_session()
print(report)

Based on the conversation, here is a summary of the user's main psychological concerns and issues that came up:

The user's main concerns seem to be dealing with their scattered mind and social anxiety, as well as managing their Attention Deficit Hyperactivity Disorder (ADHD) symptoms. Specifically, they mentioned feeling stuck in a rut, having racing thoughts, and struggling to find effective ways to manage their mental health.

Here are a few possible methods or strategies that may be helpful for the user to work on these issues:

1. Establishing a daily routine and structure can help improve focus and reduce symptoms of ADHD.
2. Engaging in regular physical activity can help improve focus, reduce symptoms of ADHD, and boost overall mental health.
3. Practicing mindfulness meditation, such as deep breathing, progressive muscle relaxation, or guided meditation, can help reduce symptoms of ADHD and improve mental health.
4. Considering Cognitive-Behavioral Therapy (CBT) to identify and